In [ ]:
using ODBC
using DataFrames
using DSWB
using Formatting

dsn = "dswb-natgeo" # Redshift esetTable(tableRt, tableType = "RESOURCE_TABLE")ndpoint
table = "beacons_4744" # beacon table name
tableRt = "beacons_4744_rt"

# Connect to Beacon Data
setRedshiftEndpoint(dsn)
setTable(tableRt, tableType = "RESOURCE_TABLE")
setTable(table)

In [ ]:
# Packages
include("../../Lib/Include-Package-v2.1.jl")

#TV = timeVariables(2017,1,2,2,30,2017,1,2,2,45);
#TV = timeVariables(2017,3,20,10,0,2017,3,20,13,59);
TV = weeklyTimeVariables(days=1);

In [ ]:
customer = "Nat Geo" 
#productPageGroup = "%"
productPageGroup = "%" # primary page group
localUrl = "%"
resourceUrl = "%ng-black-logo%"

localTable = "$(table)_Find_Any_Resource_view"
linesOutput = 250
minimumEncoded = 100000
;

In [ ]:
try
    query("""\
        create or replace view $localTable as (
            select * from $table 
                where 
                    "timestamp" between $(TV.startTimeMsUTC) and $(TV.endTimeMsUTC) and 
                    page_group ilike '$(productPageGroup)' and
                    params_u ilike '$(localUrl)' 
        )
    """)
    cnt = query("""SELECT count(*) FROM $localTable""")
    println("$localTable count is ",cnt[1,1])
catch y
    println("setupLocalTable Exception ",y)
end

In [ ]:
function resourceSummary(localTable::ASCIIString,tableRt::ASCIIString;linesOut::Int64=25,minEncoded::Int64=1000)
    
    try
        joinTables = query("""\
        select 
        $tableRt.url, $tableRt.params_u, $localTable.url,
            count(*)
        from $localTable join $tableRt
        on $localTable.session_id = $tableRt.session_id and $localTable."timestamp" = $tableRt."timestamp"
        where 
        $tableRt.url ilike '$resourceUrl'
        group by 
        $tableRt.url, $tableRt.params_u, $localTable.url
        """);

        displayTitle(chart_title = "Resource Url Pattern $(resourceUrl)", chart_info = [TV.timeString], showTimeStamp=false)
        scrubUrlToPrint(joinTables,:url,limit=150)
        beautifyDF(joinTables[1:min(linesOut,end),:])
    catch y
        println("bigTable5 Exception ",y)
    end
end 


In [ ]:
resourceSummary(localTable,tableRt;linesOut=linesOutput,minEncoded=minimumEncoded)
;
#resourceSizes(localTable,tableRt,"%css";linesOut=linesOutput,minEncoded=minimumEncoded)

In [ ]:
function resourceSummary2(localTable::ASCIIString,tableRt::ASCIIString;linesOut::Int64=25,minEncoded::Int64=1000)
    
    try
        joinTables = query("""\
        select 
        url,params_u,count(*)
        from $tableRt
        where 
          url ilike '$resourceUrl' and
          "timestamp" between $(TV.startTimeMsUTC) and $(TV.endTimeMsUTC)
        group by 
            url, params_u
        order by count(*) desc
        """);

        displayTitle(chart_title = "Raw Resource Url Pattern $(resourceUrl)", chart_info = [TV.timeString], showTimeStamp=false)
        scrubUrlToPrint(joinTables,:url,limit=150)
        beautifyDF(joinTables[1:min(linesOut,end),:])
    catch y
        println("bigTable5 Exception ",y)
    end
end 


In [ ]:
resourceSummary2(localTable,tableRt;linesOut=linesOutput,minEncoded=minimumEncoded)
;
#resourceSizes(localTable,tableRt,"%css";linesOut=linesOutput,minEncoded=minimumEncoded)

In [ ]:
function resourceSummary3(localTable::ASCIIString,tableRt::ASCIIString;linesOut::Int64=25,minEncoded::Int64=1000)
    
    try
        joinTables = query("""\
        select 
        *
        from $tableRt
        where 
          url ilike '$resourceUrl' and
          "timestamp" between $(TV.startTimeMsUTC) and $(TV.endTimeMsUTC)
        limit 3
        """);

        displayTitle(chart_title = "Raw Resource Url Pattern $(resourceUrl)", chart_info = [TV.timeString], showTimeStamp=false)
        scrubUrlToPrint(joinTables,:url,limit=150)
        beautifyDF(joinTables[1:min(linesOut,end),:])
    catch y
        println("bigTable5 Exception ",y)
    end
end 


In [ ]:
resourceSummary3(localTable,tableRt;linesOut=linesOutput,minEncoded=minimumEncoded)
;
#resourceSizes(localTable,tableRt,"%css";linesOut=linesOutput,minEncoded=minimumEncoded)

In [ ]:
function resourceSummary4(localTable::ASCIIString,tableRt::ASCIIString;linesOut::Int64=25,minEncoded::Int64=1000)
    
    try
        joinTables = query("""\
        select 
        count(*),
        avg(start_time) as "Start Time",
        avg(fetch_start) as "Fetch S",
        avg(dns_end-dns_start) as "DNS ms",
        avg(tcp_connection_end-tcp_connection_start) as "TCP ms",
        avg(request_start) as "Req S",
        avg(response_first_byte) as "Req FB",	
        avg(response_last_byte) as "Req LB",
        max(response_last_byte) as "Max Req LB",
        url, params_u,
        avg(redirect_start) as "Redirect S",
        avg(redirect_end) as "Redirect E",
        avg(secure_connection_start) as "Secure Conn S"
        from $tableRt
        where 
          url ilike '$resourceUrl' and
          "timestamp" between $(TV.startTimeMsUTC) and $(TV.endTimeMsUTC)
        group by url, params_u
        order by count(*) desc
        """);

        displayTitle(chart_title = "Raw Resource Url Pattern $(resourceUrl)", chart_info = [TV.timeString], showTimeStamp=false)
        #scrubUrlToPrint(joinTables,limit=150)
        beautifyDF(joinTables[1:min(linesOut,end),:])
    catch y
        println("bigTable5 Exception ",y)
    end
end 


In [ ]:
resourceSummary4(localTable,tableRt;linesOut=linesOutput,minEncoded=minimumEncoded)
;
#resourceSizes(localTable,tableRt,"%css";linesOut=linesOutput,minEncoded=minimumEncoded)

In [ ]:
function resourceSummary5(localTable::ASCIIString,tableRt::ASCIIString;linesOut::Int64=25,minEncoded::Int64=1000)
    
    try
        joinTables = query("""\
        select 
        (response_last_byte-start_time) as "Time Taken",
        (start_time) as "Start Time",
        (fetch_start) as "Fetch S",
        (dns_end-dns_start) as "DNS ms",
        (tcp_connection_end-tcp_connection_start) as "TCP ms",
        (request_start) as "Req S",
        (response_first_byte) as "Req FB",	
        (response_last_byte) as "Req LB",
        url, params_u,
        (redirect_start) as "Redirect S",
        (redirect_end) as "Redirect E",
        (secure_connection_start) as "Secure Conn S"
        from $tableRt
        where 
          url ilike '$resourceUrl' and
        "timestamp" between $(TV.startTimeMsUTC) and $(TV.endTimeMsUTC) and
        (response_last_byte-start_time) > 5000
        order by "Time Taken" desc
        """);

        displayTitle(chart_title = "Raw Resource Url Pattern $(resourceUrl)", chart_info = [TV.timeString], showTimeStamp=false)
        #scrubUrlToPrint(joinTables,limit=150)
        beautifyDF(joinTables[1:min(linesOut,end),:])
    catch y
        println("bigTable5 Exception ",y)
    end
end 


In [ ]:
resourceSummary5(localTable,tableRt;linesOut=linesOutput,minEncoded=minimumEncoded)
;
#resourceSizes(localTable,tableRt,"%css";linesOut=linesOutput,minEncoded=minimumEncoded)

In [ ]:
function resourceSummary6(localTable::ASCIIString,tableRt::ASCIIString;linesOut::Int64=25,minEncoded::Int64=1000)
    
    try
        joinTables = query("""\
        select 
        (response_last_byte-start_time) as "Time Taken",
        (start_time) as "Start Time",
        (fetch_start) as "Fetch S",
        (dns_end-dns_start) as "DNS ms",
        (tcp_connection_end-tcp_connection_start) as "TCP ms",
        (request_start) as "Req S",
        (response_first_byte) as "Req FB",	
        (response_last_byte) as "Req LB",
        url, params_u,
        (redirect_start) as "Redirect S",
        (redirect_end) as "Redirect E",
        (secure_connection_start) as "Secure Conn S"
        from $tableRt
        where 
          url ilike '$resourceUrl' and
        "timestamp" between $(TV.startTimeMsUTC) and $(TV.endTimeMsUTC) and
        start_time > 10000
        order by start_time desc
        """);

        displayTitle(chart_title = "Raw Resource Url Pattern $(resourceUrl)", chart_info = [TV.timeString], showTimeStamp=false)
        #scrubUrlToPrint(joinTables,limit=150)
        beautifyDF(joinTables[1:min(linesOut,end),:])
    catch y
        println("bigTable5 Exception ",y)
    end
end 


In [ ]:
resourceSummary6(localTable,tableRt;linesOut=linesOutput,minEncoded=minimumEncoded)
;
#resourceSizes(localTable,tableRt,"%css";linesOut=linesOutput,minEncoded=minimumEncoded)